In [1]:
n = 4
N=binomial(n,2)

In [2]:
#list of unordered pairs of elements in {1,...,n}
def pairs(n): return flatten([[[i+1,j+1] for j in range(i+1,n)] for i in range(n-1)],max_level=1)

#sigma acts on an unordered pair
def act(sigma, p): return sorted([sigma(p[0]),sigma(p[1])])

#sigma acts as matrix in basis on a vector v
def act_vect(g,v,basis=identity_matrix(N)): return (basis.inverse()*g.matrix()*basis)*vector(v)

#permutation in Sigma_N which induced from perm in Sigma_n
def embed(sigma,n): return perm_from_sort([act(sigma,p) for p in pairs(n)])

#find permutation which orders a list L	
def perm_from_sort(L): return Permutation([pair[0] for pair in sorted(enumerate(L, 1), key=lambda x: x[1])]).to_cycles()

#image of generators of Sigma_n under embedding into Sigma_N
def embed_gens(n): return [embed(gen,n) for gen in SymmetricGroup(n).gens()]

In [3]:
#fundamental domain given arbitrary distinct vector l and basis B
def fund_domain(center=[i for i in range(N)],basis=identity_matrix(N),br=QQ,group=MatrixGroup(identity_matrix(N))):

    #augmented matrix for half-plane ineqs
    A = [ [center[j] - act_vect(group[i],vector(center),basis)[j] for j in range(N)] for i in range(group.order())]
    b_1 = group.order()*[0]
    bA = matrix(br,b_1).transpose().augment(matrix(br,A))

    #augmented matrix for positivity ineqs
    pos = basis.inverse().transpose()
    b_2 = [0 for i in range(N)]
    bPos = matrix(br,b_2).transpose().augment(pos)

    #augmented matrix for fund. domain
    aug_l = bA.stack(bPos)
    
    #find convex polyhedral region, i.e. intersection of all ineqs
    poly = Polyhedron(ieqs=aug_l,base_ring=br)

    return([aug_l,poly])

In [4]:
#slice the fundamental domain with the plane x_1 + ... + x_N = 1
def cross_section(region,slice_dir=[1 for i in range(N)],chi=1,br=None):
    if br is None: br=region.base_ring()
    return Polyhedron(ieqs=region.inequalities(),eqns=[[-1]+slice_dir],base_ring=br)

In [5]:
#find fixed points under G in polyhedron F to include as new vertices
#find intersection of fixed point subspace with F as polyhedron, and return those vertices
def fixed_verts(F,br=QQ):
    new_verts=set()
    for g in G:
        #get equations for fixed point subspace
        A=g.matrix() #matrix associated to group element g
        B=A-identity_matrix(N) #eqns defining fixed pt subspace are (A-Id)x == 0
        b = N*[0] #0 vector
        bB = matrix(br,b).transpose().augment(matrix(br,B))#form augmented matrix for equations defining subspace
        #get equations for intersection
        eqns=matrix(br,[list(eq) for eq in F.equations()]) #list of equations defining F
        eqns=eqns.stack(bB) #augmented matrix including fixed point subspace equations
        #get list of inequalities for fundamental domain as convex polyhedron
        ieqs=matrix(br,[list(ieq) for ieq in F.inequalities()])
        #form intersection of fund_domain with fixed point subspace
        intersection_F_fixed_pt_subspace=Polyhedron(ieqs=ieqs,eqns=eqns)
        verts=intersection_F_fixed_pt_subspace.vertices()
        for vert in verts:
            new_verts.add(tuple(vert))
    return list(new_verts)

In [6]:
#define the finite group
G = PermutationGroup(embed_gens(n)) #symmetric group \sigma_n as a subgroup of \sigma_N, with 2 generators
#G = MatrixGroup(matrix(QQ,[[0,1,0],[1,0,0],[0,0,1]]))

In [7]:
#compute fundamental domain to get list of vertices
F=fund_domain(group=G); F[1]

A 6-dimensional polyhedron in QQ^6 defined as the convex hull of 1 vertex and 7 rays (use the .plot() method to plot)

In [8]:
F2=cross_section(F[1])
[tuple(v) for v in F2.vertices()]

[(1/2, 0, 0, 0, 0, 1/2),
 (0, 1/2, 0, 0, 1/2, 0),
 (0, 0, 0, 0, 1/2, 1/2),
 (0, 0, 0, 1/3, 1/3, 1/3),
 (0, 0, 1/2, 1/2, 0, 0),
 (0, 0, 1/3, 0, 1/3, 1/3),
 (0, 0, 0, 0, 0, 1)]

In [9]:
vertices=fixed_verts(F2,QQ); vertices

[(0, 1/4, 1/4, 1/4, 1/4, 0),
 (1/2, 0, 0, 0, 0, 1/2),
 (0, 0, 1/3, 0, 1/3, 1/3),
 (1/4, 0, 1/4, 1/4, 0, 1/4),
 (1/4, 1/4, 0, 0, 1/4, 1/4),
 (0, 0, 0, 1/3, 1/3, 1/3),
 (0, 0, 1/2, 1/2, 0, 0),
 (0, 0, 0, 0, 0, 1),
 (1/6, 1/6, 1/6, 1/6, 1/6, 1/6),
 (0, 0, 0, 0, 1/2, 1/2),
 (0, 1/2, 0, 0, 1/2, 0)]

In [10]:
#determine if face1 is glued to face2 by the action of G
#faces are given by list or set of vertex indices
def faces_glued(face1,face2,G,vertices):
    for g in G:
        g_face1 = [vertices.index(tuple(act_vect(g,vertices[i]))) if tuple(act_vect(g,vertices[i])) in vertices else i for i in face1]
        if set(g_face1) == set(face2):
            return True
    return False

In [11]:
#return all elements which glue face1 to face2
#faces are given by list or set of vertex indices
def gluing_elmts(face1,face2,G,vertices):
    gluing_elements = []
    for g in G:
        #act on vertices defining face. if g.v_i is not in vertex set {v_i}, use v_i. 
        g_face1 = [vertices.index(tuple(act_vect(g,vertices[i]))) if tuple(act_vect(g,vertices[i])) in vertices else i for i in face1]
        #if vertex sets match, add element to list
        if set(g_face1) == set(face2):
            gluing_elements.append(g)
    return gluing_elements

In [12]:
def trivial_self_gluing(face,G,vertices):
    trivial = True
    for g in G:
        #act on vertices defining face
        g_face = [vertices.index(tuple(act_vect(g,vertices[i]))) if tuple(act_vect(g,vertices[i])) in vertices else i for i in face]
        #if vertex sets match, add element to list
        if set(g_face) == set(face):
            #check if the gluing is non-trivial
            if [g_face.index(i) for i in face] != [i for i in range(len(face))]:
                trivial = False
    return trivial

In [13]:
#toss out any facets with non-trivial self-gluings such as [0,3,4] or [1,6]
#build list of faces for each dim
import itertools
def faces(k): return [face for face in itertools.combinations(range(len(vertices)), k+1) if trivial_self_gluing(face,G,vertices)]
len(faces(2))

127

In [14]:
#find classes of vertices which are glued
def glued_faces(k):
    glued_verts = []
    for i in faces(k):
        found = False
        for equiv_class in glued_verts:
            if len(equiv_class) >= 1:
                if faces_glued(i,equiv_class[0],G,vertices):
                    equiv_class.append(i)
                    found=True
        if not found:
            glued_verts.append([i])
    return glued_verts

In [15]:
#compute glued 0-faces
zero_faces=glued_faces(0)

In [16]:
#compute glued 1-faces
one_faces=glued_faces(1)

In [87]:
two_faces=glued_faces(2)

In [90]:
three_faces=glued_faces(3)

In [92]:
four_faces=glued_faces(4)

In [93]:
five_faces=glued_faces(5)

In [ ]:
six_faces=glued_faces(6)

In [96]:
[len(zero_faces),len(one_faces),len(two_faces),len(three_faces),len(four_faces),len(five_faces)]

[7, 22, 42, 56, 56, 42]

In [164]:
[len(faces(k)) for k in range(3)]

[3, 3, 1]

In [97]:
7-22+42-56+56-42

-15

In [100]:
print("0-faces:",zero_faces)
print("1-faces:",one_faces)
print("2-faces:",two_faces)
print("3-faces:",three_faces)
print("4-faces:",four_faces)
print("5-faces:",five_faces)

0-faces: [[(0,), (3,), (4,)], [(1,), (6,), (10,)], [(2,)], [(5,)], [(7,)], [(8,)], [(9,)]]
1-faces: [[(0, 1), (3, 10), (4, 6)], [(0, 2), (2, 3), (2, 4)], [(0, 5), (3, 5), (4, 5)], [(0, 6), (0, 10), (1, 3), (1, 4), (3, 6), (4, 10)], [(0, 7), (3, 7), (4, 7)], [(0, 8), (3, 8), (4, 8)], [(0, 9), (3, 9), (4, 9)], [(1, 2), (2, 6), (2, 10)], [(1, 5), (5, 6), (5, 10)], [(1, 7), (6, 7), (7, 10)], [(1, 8), (6, 8), (8, 10)], [(1, 9), (6, 9), (9, 10)], [(2, 5)], [(2, 7)], [(2, 8)], [(2, 9)], [(5, 7)], [(5, 8)], [(5, 9)], [(7, 8)], [(7, 9)], [(8, 9)]]
2-faces: [[(0, 1, 2), (2, 3, 10), (2, 4, 6)], [(0, 1, 3), (0, 1, 4), (0, 3, 10), (0, 4, 6), (3, 4, 6), (3, 4, 10)], [(0, 1, 5), (3, 5, 10), (4, 5, 6)], [(0, 1, 6), (0, 1, 10), (1, 3, 10), (1, 4, 6), (3, 6, 10), (4, 6, 10)], [(0, 1, 7), (3, 7, 10), (4, 6, 7)], [(0, 1, 8), (3, 8, 10), (4, 6, 8)], [(0, 1, 9), (3, 9, 10), (4, 6, 9)], [(0, 2, 5), (2, 3, 5), (2, 4, 5)], [(0, 2, 6), (0, 2, 10), (1, 2, 3), (1, 2, 4), (2, 3, 6), (2, 4, 10)], [(0, 2, 7), (2, 3,

In [102]:
len(faces(2))

127

In [81]:
faces_glued(one_faces[1][0],one_faces[1][0],G,vertices)

True

In [84]:
trivial_self_gluing(one_faces[1][2],G,vertices)

True

In [18]:
#TODO: compute boundary of each face
#replace vertices with glued_verts, keep track of orientation
def face_boundary(face):
    boundary = []
    for i in face:
        face_remove_i = list(face)
        face_remove_i.remove(i)
        boundary.append(face_remove_i)
    return boundary